In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import itertools
from scipy.optimize import curve_fit

random.seed(4)
# Reducing the size of the network

df_edges = pd.read_csv("com-Amazon.csv", delimiter = " ")
OG = nx.from_pandas_edgelist(df_edges, source="From", target="To", create_using= nx.DiGraph) #the original co-purchasing network
edges = random.sample(list(OG.nodes()), int(OG.number_of_nodes() * 0.26)) # will use only 25% of the graph
G=OG.subgraph(edges)
largest_cc = max(nx.weakly_connected_components(G), key=len) #find the largest weakly component
G1=G.subgraph(largest_cc) # the final graph with the largest weakly connected component
G2= G1.to_undirected()
G2_node_list= [] # add node that have the degree > 2
for node,degree in G1.degree():
    if degree > 2:
        G2_node_list.append(node)
G2= G1.subgraph(G2_node_list) #creating the graph base on the degree > 2
        
removed_edges = random.sample(list(G2.edges()), int(G2.number_of_edges() * 0.30)) #we will removed 20% of the edges
G_train = G2.copy()
G_train.remove_edges_from(removed_edges)
G_test = G2.copy()
#G_test.add_edges_from(removed_edges)

G_train=G_train.to_undirected()
print(G_train)


Graph with 5465 nodes and 4991 edges


In [2]:
print("Density:", nx.density(G2))
print("Average Clustering:",nx.average_clustering(G2))
print("Transitivity:",nx.transitivity(G2))
#print("Connected Components in Training Set:",nx.number_connected_components(G_train))

degree_list=[]
for degree in list(G2.degree()):
    degree_list.append(degree[1])

print("Average Degree:",sum(degree_list)/len(degree_list))



Density: 0.00023877490057185418
Average Clustering: 0.17441234116408738
Transitivity: 0.18207730812013348
Average Degree: 2.6093321134492222


In [3]:
# jacard
n_star= np.arange(10,110,10)

# Calculating for Jacard value
def JA_cal(graph): 
    jacard = nx.jaccard_coefficient(graph)
    jacard_pred = []
    for u, v, p in jacard:
        jacard_pred.append([u, v, p])
    jacard_pred = pd.DataFrame(jacard_pred, columns=['u', 'v', 'p']) 
    jacard_pred = jacard_pred.sort_values(by='p', ascending=False)
    pred= jacard_pred
    return pred

# Calculating for Adamic Adar value

def AA_cal(graph):
    adamic = nx.adamic_adar_index(graph)
    adamic_pred = []
    for u, v, p in adamic:
        adamic_pred.append([u, v, p])
    adamic_pred = pd.DataFrame(adamic_pred, columns=['u', 'v', 'p'])
    adamic_pred = adamic_pred.sort_values(by='p', ascending=False)
    pred= adamic_pred
    return pred

# Calculating for Preferential attachment value

def PA_cal(graph):
    pref = nx.preferential_attachment(graph)
    PA_prec= []
    for u, v, p in pref:
        PA_prec.append([u, v, p])
    PA_prec = pd.DataFrame(PA_prec, columns=['u', 'v', 'p'])
    PA_prec = PA_prec.sort_values(by='p', ascending=False)
    pred= PA_prec
    return pred

# Calculating for Resource Allocation value

def RA_cal(graph):
    res = nx.resource_allocation_index(graph)
    RA_prec= []
    for u, v, p in res:
        RA_prec.append([u, v, p])
    RA_prec = pd.DataFrame(RA_prec, columns=['u', 'v', 'p'])
    RA_prec = RA_prec.sort_values(by='p', ascending=False)
    pred= RA_prec
    return pred

def HDI(graph): #Hub Depressed Index
    comb=[]
    for t in list(itertools.product(list(G_train.nodes()),(G_train.nodes()))):    #all combinations of nodes, excludes self loops
        if t[0]!=t[1]:
            comb.append(t)
    hub_promoted_index=[]
    for k in comb:
        if len(sorted(nx.common_neighbors(G_train,k[0],k[1])))!=0 and (k[0],k[1]) not in list(G_train.edges()) and (k[1],k[0]) not in list(G_train.edges()): 
            hub_promoted_index.append( (k[0], k[1], (len(sorted(nx.common_neighbors(G_train,k[0],k[1]))))/max( G_train.degree[k[0]] , G_train.degree[k[1]]) ))
    hub_promoted_index = pd.DataFrame(hub_promoted_index, columns=['u', 'v', 'p']) 
    hub_promoted_index.sort_values(by='p', ascending=False)
    return hub_promoted_index

def HPI(graph): #Hub Promoted Index
    comb=[]
    for t in list(itertools.product(list(G_train.nodes()),(G_train.nodes()))):    #all combinations of nodes, excludes self loops
        if t[0]!=t[1]:
            comb.append(t)
    hub_promoted_index=[]
    for k in comb:
        if len(sorted(nx.common_neighbors(G_train,k[0],k[1])))!=0 and (k[0],k[1]) not in list(G_train.edges()) and (k[1],k[0]) not in list(G_train.edges()): 
            hub_promoted_index.append( (k[0], k[1], (len(sorted(nx.common_neighbors(G_train,k[0],k[1]))))/min( G_train.degree[k[0]] , G_train.degree[k[1]]) ))
    hub_promoted_index = pd.DataFrame(hub_promoted_index, columns=['u', 'v', 'p']) 
    hub_promoted_index.sort_values(by='p', ascending=False)
    return hub_promoted_index

def LHNI(graph): #Leicht-Holme-Newman Index
    comb=[]
    for t in list(itertools.product(list(G_train.nodes()),(G_train.nodes()))):    #all combinations of nodes, excludes self loops
        if t[0]!=t[1]:
            comb.append(t)
    hub_promoted_index=[]
    for k in comb:
        if len(sorted(nx.common_neighbors(G_train,k[0],k[1])))!=0 and (k[0],k[1]) not in list(G_train.edges()) and (k[1],k[0]) not in list(G_train.edges()): 
            hub_promoted_index.append( (k[0], k[1], (len(sorted(nx.common_neighbors(G_train,k[0],k[1]))))/(G_train.degree[k[0]] * G_train.degree[k[1]]) ))
    hub_promoted_index = pd.DataFrame(hub_promoted_index, columns=['u', 'v', 'p']) 
    hub_promoted_index.sort_values(by='p', ascending=False)
    return hub_promoted_index
    
# Check to see if all of the link prediction is accurately predicting the value
def pred_accuracy(predEdges,removed_edges,k):
    correctly_nodes = [value for value in predEdges if value in removed_edges]
    number_of_correct= len(correctly_nodes)
    rate= number_of_correct/k
    return (number_of_correct,rate)
    
def FinalValue(graph, link):  #input training graph
    Ks = np.arange(10,310,10) # starting from 10, start incrementing by 10 -> 10,20,30,...
    if link == "JA":
        pred= JA_cal(graph)
    elif link == 'AA':
        pred= AA_cal(graph)
    elif link == 'PA':
        pred= PA_cal(graph)
    elif link == 'RA':
        pred= RA_cal(graph)
    elif link == "HDI":
        pred=HDI(graph)
    elif link == "HPI":
        pred=HPI(graph)
    elif link == "LHNI":
        pred=LHNI(graph)

    
    print("For {} ".format(link))
    Final_Value= []
    for k in Ks:     
        predEdges= []
        score= []
        for i in range(k):
            predEdges.append((int(pred.iloc[i]['u']),int(pred.iloc[i]['v'])))
            score.append(((pred.iloc[i]['p'])))
        result = pred_accuracy(predEdges,removed_edges,k)
        Final_Value.append(result)
        print("for {}, the number of correct edges being predicted {} and the rate of it is {}".format(k,result[0],result[1]))
    return Final_Value


FinalValue(G_train, link= "JA")
FinalValue(G_train, link= "AA")
FinalValue(G_train, link= "PA")
FinalValue(G_train, link= "RA")
FinalValue(G_train,link= "HDI")
FinalValue(G_train,link= "HPI")




For JA 
for 10, the number of correct edges being predicted 1 and the rate of it is 0.1
for 20, the number of correct edges being predicted 2 and the rate of it is 0.1
for 30, the number of correct edges being predicted 2 and the rate of it is 0.06666666666666667
for 40, the number of correct edges being predicted 3 and the rate of it is 0.075
for 50, the number of correct edges being predicted 4 and the rate of it is 0.08
for 60, the number of correct edges being predicted 7 and the rate of it is 0.11666666666666667
for 70, the number of correct edges being predicted 7 and the rate of it is 0.1
for 80, the number of correct edges being predicted 8 and the rate of it is 0.1
for 90, the number of correct edges being predicted 10 and the rate of it is 0.1111111111111111
for 100, the number of correct edges being predicted 11 and the rate of it is 0.11
for 110, the number of correct edges being predicted 11 and the rate of it is 0.1
for 120, the number of correct edges being predicted 12 

[(2, 0.2),
 (2, 0.1),
 (2, 0.06666666666666667),
 (2, 0.05),
 (2, 0.04),
 (2, 0.03333333333333333),
 (2, 0.02857142857142857),
 (2, 0.025),
 (2, 0.022222222222222223),
 (2, 0.02),
 (4, 0.03636363636363636),
 (6, 0.05),
 (6, 0.046153846153846156),
 (6, 0.04285714285714286),
 (7, 0.04666666666666667),
 (7, 0.04375),
 (7, 0.041176470588235294),
 (8, 0.044444444444444446),
 (8, 0.042105263157894736),
 (9, 0.045),
 (10, 0.047619047619047616),
 (10, 0.045454545454545456),
 (10, 0.043478260869565216),
 (10, 0.041666666666666664),
 (11, 0.044),
 (12, 0.046153846153846156),
 (12, 0.044444444444444446),
 (13, 0.04642857142857143),
 (13, 0.04482758620689655),
 (14, 0.04666666666666667)]